In [10]:
import numpy as np
import pandas as pd
from feast.dqm.profilers.ge_profiler import ge_profiler

from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations.dataset import PandasDataset

from feast.infra.offline_stores.file import SavedDatasetFileStorage

import feast

In [6]:
store = feast.FeatureStore("feature_repo/")
store


In [7]:
entity_df = pd.read_parquet('feature_repo/raw_data/loan_table.parquet')
entity_df

,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp
0,10000,19530219_5179,76104,22,59000,RENT,123.0,PERSONAL,35000,16.02,1,2021-08-25 20:34:41.361000+00:00,2021-08-25 20:34:41.361000+00:00
1,10001,19520816_8737,70380,21,9600,OWN,5.0,EDUCATION,1000,11.14,0,2021-08-25 20:16:20.128000+00:00,2021-08-25 20:16:20.128000+00:00
2,10002,19860413_2537,97039,25,9600,MORTGAGE,1.0,MEDICAL,5500,12.87,1,2021-08-25 19:57:58.896000+00:00,2021-08-25 19:57:58.896000+00:00
3,10003,19760701_8090,63785,23,65500,RENT,4.0,MEDICAL,35000,15.23,1,2021-08-25 19:39:37.663000+00:00,2021-08-25 19:39:37.663000+00:00
4,10004,19830125_8297,82223,24,54400,RENT,8.0,MEDICAL,35000,14.27,1,2021-08-25 19:21:16.430000+00:00,2021-08-25 19:21:16.430000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28633,38633,19491126_1487,43205,57,53000,MORTGAGE,1.0,PERSONAL,5800,13.16,0,2020-08-25 21:48:06.292000+00:00,2020-08-25 21:48:06.292000+00:00
28634,38634,19681208_6537,24872,54,120000,MORTGAGE,4.0,PERSONAL,17625,7.49,0,2020-08-25 21:29:45.059000+00:00,2020-08-25 21:29:45.059000+00:00
28635,38635,19880422_2592,68826,65,76000,RENT,3.0,HOMEIMPROVEMENT,35000,10.99,1,2020-08-25 21:11:23.826000+00:00,2020-08-25 21:11:23.826000+00:00
28636,38636,19901017_6108,92014,56,150000,MORTGAGE,5.0,PERSONAL,15000,11.48,0,2020-08-25 20:53:02.594000+00:00,2020-08-25 20:53:02.594000+00:00


In [14]:
job = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('features_group_v1')
)

store.create_saved_dataset(
    from_=job,
    name='my_training_ds',
    storage=SavedDatasetFileStorage(path='my_training_ds.parquet')
)

/opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.9/site-packages/feast/feature_store.py:1185: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


<SavedDataset(name = my_training_ds, features = ['credit_history:credit_card_due', 'credit_history:mortgage_due', 'credit_history:student_loan_due', 'credit_history:vehicle_loan_due', 'credit_history:hard_pulls', 'credit_history:missed_payments_2y', 'credit_history:missed_payments_1y', 'credit_history:missed_payments_6m', 'credit_history:bankruptcies', 'zipcode_features:city', 'zipcode_features:state', 'zipcode_features:location_type', 'zipcode_features:tax_returns_filed', 'zipcode_features:population', 'zipcode_features:total_wages'], join_keys = ['zipcode', 'created_timestamp', 'person_age', 'person_emp_length', 'loan_status', 'person_home_ownership', 'person_income', 'dob_ssn', 'loan_id', 'loan_int_rate', 'loan_intent', 'loan_amnt'], storage = <feast.infra.offline_stores.file_source.SavedDatasetFileStorage object at 0x28903ff40>, full_feature_names = False, tags = {}, feature_service_name = None, _retrieval_job = <feast.infra.offline_stores.file.FileRetrievalJob object at 0x2886f325

In [15]:
ds = store.get_saved_dataset('my_training_ds')
ds.to_df()

/opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.9/site-packages/feast/feature_store.py:1238: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


FeastJoinKeysDuringMaterialization: The DataFrame from my_training_ds.parquet being materialized must have at least {'zipcode', 'created_timestamp', 'person_age', 'person_emp_length', 'loan_status', 'person_home_ownership', 'person_income', 'dob_ssn', 'loan_id', 'loan_int_rate', 'event_timestamp', 'loan_intent', 'loan_amnt'} columns present, but these were missing: {'created_timestamp'} 

In [16]:
pd.read_parquet('my_training_ds.parquet')

,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,...,missed_payments_2y,missed_payments_1y,missed_payments_6m,bankruptcies,city,state,location_type,tax_returns_filed,population,total_wages
0,38637,19960703_3449,69033,66,42000,RENT,2.0,MEDICAL,6475,9.99,...,4,0,1,0,IMPERIAL,NE,PRIMARY,1171,2141,32596076
12,38276,19770128_3701,61345,39,75000,RENT,0.0,PERSONAL,14400,12.53,...,6,1,0,0,NEPONSET,IL,PRIMARY,413,730,11569469
1,38287,19600201_7821,54901,48,140000,MORTGAGE,4.0,EDUCATION,18000,9.99,...,1,0,0,0,OSHKOSH,WI,PRIMARY,15059,23996,469695341
2,38286,19921119_7451,20912,36,58000,RENT,18.0,DEBTCONSOLIDATION,5000,6.62,...,0,0,0,0,TAKOMA PARK,MD,PRIMARY,11982,19175,543954725
3,38285,19580615_7535,64856,46,48956,RENT,0.0,MEDICAL,2800,10.59,...,1,1,0,1,PINEVILLE,MO,PRIMARY,1609,2929,45969820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28632,10041,20000903_5182,60191,26,300000,MORTGAGE,10.0,VENTURE,20000,7.88,...,1,0,0,0,WOOD DALE,IL,PRIMARY,7487,12973,294594830
28633,10153,19610414_1854,81008,25,61959,RENT,9.0,MEDICAL,25000,12.73,...,1,1,0,0,PUEBLO,CO,PRIMARY,5074,8861,172125946
28634,10018,19751208_7623,20692,23,113000,RENT,8.0,DEBTCONSOLIDATION,35000,18.25,...,4,1,0,0,VALLEY LEE,MD,PRIMARY,589,1054,28617466
28624,10188,19711230_6002,13602,24,75000,RENT,4.0,EDUCATION,25000,10.25,...,1,3,0,0,FORT DRUM,NY,PRIMARY,2360,2867,47053802
